In [38]:
import numpy as np

class Matrix:
    def __init__(self, raw):
        self.raw = raw
        self.rows = len(raw)
        self.cols = len(raw[0])

    @staticmethod
    def GetIdentityMatrix(n):
        return [[1 if i == j else 0 for j in range(n)] for i in range(n)]

    def defineElement(self, n, applier):
        row = n // self.cols
        col = n % self.cols
        self.raw[row][col] = applier


class OperableMatrix(Matrix):
    def Transpose(self, ShouldChange=False):
        output = [list(row) for row in zip(*self.raw)]
        if ShouldChange:
            self.raw = output
        return OperableMatrix(output)

    def ScalarMultiplication(self, scalar, ShouldChange=False):
        output = [[el * scalar for el in row] for row in self.raw]
        if ShouldChange:
            self.raw = output
        return OperableMatrix(output)

    def MatrixMultiplication(self, matrix2, matrix1=None):
        matrix1 = matrix1 or self
        if matrix1.cols != matrix2.rows:
            raise ValueError("Dimensiones incompatibles para multiplicación de matrices")
        result = [
            [sum(matrix1.raw[i][k] * matrix2.raw[k][j] for k in range(matrix1.cols))
             for j in range(matrix2.cols)]
            for i in range(matrix1.rows)
        ]
        return OperableMatrix(result)

    def Determinant(self, matrix=None):
        matrix = matrix or self.raw
        if len(matrix) != len(matrix[0]):
            raise ValueError("Debe ser una matriz cuadrada")
        if len(matrix) == 1:
            return matrix[0][0]
        if len(matrix) == 2:
            return matrix[0][0]*matrix[1][1] - matrix[0][1]*matrix[1][0]

        det = 0
        for col in range(len(matrix[0])):
            minor = [row[:col] + row[col+1:] for row in matrix[1:]]
            det += ((-1) ** col) * matrix[0][col] * self.Determinant(minor)
        return det

    def Inverse(self):
        det = self.Determinant()
        if det == 0:
            raise ValueError("La matriz no es invertible (det=0)")

        n = self.rows
        cofactors = []
        for i in range(n):
            row = []
            for j in range(n):
                minor = [r[:j] + r[j+1:] for k, r in enumerate(self.raw) if k != i]
                sign = 1 if (i + j) % 2 == 0 else -1
                row.append(sign * self.Determinant(minor))
            cofactors.append(row)

        adjugate = OperableMatrix(cofactors).Transpose().raw
        return OperableMatrix(adjugate).ScalarMultiplication(1 / det)


# Diccionario
diccionario = {
    'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9,
    'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18,
    'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, ' ': 26
}

# Modular
def inverso_modular(a, m):
    a = (a % m + m) % m
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return None

def moduleIt(n, m=27):
    return (n % m + m) % m

def Module27(matrix):
    return [[moduleIt(el) for el in row] for row in matrix]

def Decode(matrix):
    output = []
    for i in range(len(matrix[0])):
        for row in matrix:
            val = int(round(row[i]))
            output.append(next(k for k, v in diccionario.items() if v == val))
    return output

def TextToArray(text):
    text = text.upper()
    if len(text) % n > 0:
        text += " " * (n - len(text) % n)
    return [diccionario[char] for char in text]

def ArrayToMatrix(array):
    output = [[] for _ in range(n)]
    for u in range(len(array) // n):
        for i in range(n):
            output[i].append(array[u * n + i])
    return output

def UNC_TextToMatrix(text):
    output = [diccionario[char] for char in text]
    return ArrayToMatrix(output)

# Configuración principal
n = 4
mensaje = "Consistent effort and practice are essential for everything"

c = [
    [1, 0, 0, 0],
    [2, 1, 0, 0],
    [3, 4, 1, 0],
    [5, 6, 7, 1]
]

c = OperableMatrix(c)

# Validaciones
if c.rows != c.cols:
    print("...No es cuadrada")
if c.Determinant() == 0:
    print("...La matriz no es invertible")

In [27]:
# Mensaje original
mensaje

'Consistent effort and practice are essential for everything'

In [28]:
# --- Codificación ---

# Carácteres codificados
codificados = TextToArray(mensaje)
print(codificados)

[2, 14, 13, 18, 8, 18, 19, 4, 13, 19, 26, 4, 5, 5, 14, 17, 19, 26, 0, 13, 3, 26, 15, 17, 0, 2, 19, 8, 2, 4, 26, 0, 17, 4, 26, 4, 18, 18, 4, 13, 19, 8, 0, 11, 26, 5, 14, 17, 26, 4, 21, 4, 17, 24, 19, 7, 8, 13, 6, 26]


In [29]:
# Matriz de carácteres codificados
A = OperableMatrix(ArrayToMatrix(codificados))
(np.array(A.raw))

array([[ 2,  8, 13,  5, 19,  3,  0,  2, 17, 18, 19, 26, 26, 17,  8],
       [14, 18, 19,  5, 26, 26,  2,  4,  4, 18,  8,  5,  4, 24, 13],
       [13, 19, 26, 14,  0, 15, 19, 26, 26,  4,  0, 14, 21, 19,  6],
       [18,  4,  4, 17, 13, 17,  8,  0,  4, 13, 11, 17,  4,  7, 26]])

In [30]:
# Matriz codificada (Multiplicada)
matriz_codificada = c.MatrixMultiplication(A).raw
(np.array(matriz_codificada))

array([[  2,   8,  13,   5,  19,   3,   0,   2,  17,  18,  19,  26,  26,
         17,   8],
       [ 18,  34,  45,  15,  64,  32,   2,   8,  38,  54,  46,  57,  56,
         58,  29],
       [ 75, 115, 141,  49, 161, 128,  27,  48,  93, 130,  89, 112, 115,
        166,  82],
       [203, 285, 365, 170, 264, 293, 153, 216, 295, 239, 154, 275, 305,
        369, 186]])

In [31]:
# Matriz en módulo 27
matriz_codificada_mod = Module27(matriz_codificada)
(np.array(matriz_codificada_mod))

array([[ 2,  8, 13,  5, 19,  3,  0,  2, 17, 18, 19, 26, 26, 17,  8],
       [18,  7, 18, 15, 10,  5,  2,  8, 11,  0, 19,  3,  2,  4,  2],
       [21,  7,  6, 22, 26, 20,  0, 21, 12, 22,  8,  4,  7,  4,  1],
       [14, 15, 14,  8, 21, 23, 18,  0, 25, 23, 19,  5,  8, 18, 24]])

In [32]:
# Mensaje codificado
mensaje_codificado = ''.join(Decode(matriz_codificada_mod))
(mensaje_codificado)

'CSVOIHHPNSGOFPWITK VDFUXACASCIVARLMZSAWXTTIT DEF CHIREESICBY'

In [33]:
# Codificado a matriz
matrixed = OperableMatrix(UNC_TextToMatrix(mensaje_codificado))
(np.array(matrixed.raw))

array([[ 2,  8, 13,  5, 19,  3,  0,  2, 17, 18, 19, 26, 26, 17,  8],
       [18,  7, 18, 15, 10,  5,  2,  8, 11,  0, 19,  3,  2,  4,  2],
       [21,  7,  6, 22, 26, 20,  0, 21, 12, 22,  8,  4,  7,  4,  1],
       [14, 15, 14,  8, 21, 23, 18,  0, 25, 23, 19,  5,  8, 18, 24]])

In [34]:
# Inversa de C
inversedC = c.Inverse()
(np.array(inversedC.raw))

array([[  1.,   0.,   0.,   0.],
       [ -2.,   1.,   0.,   0.],
       [  5.,  -4.,   1.,   0.],
       [-28.,  22.,  -7.,   1.]])

In [35]:
# Módulo 27 de la inversa
inversedC_mod = OperableMatrix(Module27(inversedC.raw))
(np.array(inversedC_mod.raw))

array([[ 1.,  0.,  0.,  0.],
       [25.,  1.,  0.,  0.],
       [ 5., 23.,  1.,  0.],
       [26., 22., 20.,  1.]])

In [36]:
# Multiplicación de matrices
multiplicado = inversedC_mod.MatrixMultiplication(matrixed).raw
multiplicado_mod = Module27(multiplicado)
(np.array(multiplicado_mod))

array([[ 2.,  8., 13.,  5., 19.,  3.,  0.,  2., 17., 18., 19., 26., 26.,
        17.,  8.],
       [14., 18., 19.,  5., 26., 26.,  2.,  4.,  4., 18.,  8.,  5.,  4.,
        24., 13.],
       [13., 19., 26., 14.,  0., 15., 19., 26., 26.,  4.,  0., 14., 21.,
        19.,  6.],
       [18.,  4.,  4., 17., 13., 17.,  8.,  0.,  4., 13., 11., 17.,  4.,
         7., 26.]])

In [37]:
# Mensaje decodificado
mensaje_decodificado = ''.join(Decode(multiplicado_mod))
(mensaje_decodificado)

'CONSISTENT EFFORT AND PRACTICE ARE ESSENTIAL FOR EVERYTHING '